In [4]:
import pandas as pd
from ortools.linear_solver import pywraplp

data = pd.read_csv("../Downloads/truc/faces.utf8.csv")
data = data.values
briefs = pd.read_csv("../Downloads/truc/briefs.utf8.csv")
briefs = briefs.values

def get_week(week, data):
    array = []
    for i in range(0, len(data)):
        if data[i][0] == week:
            array.append(data[i])
    return (array)

dict_col = {}
dict_col["25_49csp"] = 12
dict_col["cspp"] = 15
dict_col["urb"] = 18
dict_col["all"] = 14
dict_col["f25_49rda"] = 17
dict_col["f25_49csp"] = 16
dict_col["15_24"] = 10
dict_col["25_49"] = 11
dict_col["25_49urb"] = 13

def get_max_week():
    return (briefs[len(briefs)-1][0])

def solve_week(week_nb):
    brief = get_week(week_nb, briefs)
    solver = pywraplp.Solver('SolveStigler', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

    faces = [[]] * len(data)
    objective = solver.Objective()
    for i in range(0, len(data)):
        faces[i] = solver.NumVar(0, 1, data[i][0])
        objective.SetCoefficient(faces[i], 1)
    objective.SetMinimization()

    constraints = [0] * len(brief)
    for i in range(0, len(brief)):
        constraints[i] = solver.Constraint(brief[i][3], solver.infinity())
        for j in range(0, len(data)):
            contactkey = dict_col[brief[i][2]]
            constraints[i].SetCoefficient(faces[j], data[j][contactkey])

    status = solver.Solve()
    if status == solver.OPTIMAL:
        print('Optimal Solution Found')
        nb_faces = 0
        for i in range (0, len(data)):
            if (faces[i].solution_value() > 0):
                nb_faces += 1
        print(nb_faces)
    else:
        if status == solver.FEASIBLE:
            print('Solution Found, but not Optimal')
        else:
            print('The solver could not solve the problem')

for i in range(1, get_max_week() + 1):
    solve_week(i)

Optimal Solution Found
1062
Optimal Solution Found
1030
Optimal Solution Found
1274
The solver could not solve the problem
Optimal Solution Found
865
Optimal Solution Found
999
